# Template

In [25]:
import re, os, sys, pickle, pickle
from pathlib import Path
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from sklearn.decomposition import PCA

# my own libaries
from GenomicWindows import window
import GenomicIntervals

np.random.seed(7)

Parallel computing:

In [26]:
# import ipyparallel
# from ipyparallel import depend, require

# # create client & view
# rc = ipyparallel.Client()
# dview = rc[:]
# bview = rc.load_balanced_view() # you can set block=True to have this apply globally

# # scatter 'id', so id=0,1,2 on engines 0,1,2
# dview.scatter('node_id', rc.ids, flatten=True)
# print("Engine IDs: ", dview['node_id'])
# # create a Reference to `id`. This will be a different value on each engine
# ref = ipyparallel.Reference('node_id')

# with dview.sync_imports(): # imports libs across dview
#     import re, os, sys, pickle
#     from pathlib import Path
#     import numpy
#     import pandas
#     from pandas import DataFrame
    
# dview.execute('numpy.random.seed(7)')

# def scatter_func(f):
#     dview.scatter(f.__name__, [f]*len(bview), flatten=True)    

# def getmem():
#     p = subprocess.Popen("echo $HOSTNAME", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#     stdout, stderr = p.communicate()
#     return node_id, stdout.decode().strip(), resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024

# def print_mem():
#     for node_id, host_name, mem in  dview.apply(getmem).get():
#         print("{}{: >10}{: 10.2f} Mb". format(node_id, host_name, mem))

# def parallel_apply(grouped_df, fun):
#     return pandas.concat(dview.map_sync(fun, (group for name, group in grouped_df)))

Plotting setup:

In [27]:
%matplotlib inline

# Make inline plots vector graphics instead of raster graphics
from IPython.display import set_matplotlib_formats
#set_matplotlib_formats('pdf', 'svg')
set_matplotlib_formats('retina', 'png')

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.patches import Rectangle
from mpl_toolkits.basemap import Basemap
#matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)

import mpld3

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_context("paper")

# lowess for plotting
from statsmodels.nonparametric.smoothers_lowess import lowess

def stairs(df, start='start', end='end', pos='pos', endtrim=0):
    "Turn a df with start, end into one with pos to plot as stairs"
    df1 = df.copy(deep=True)
    df2 = df.copy(deep=True)
    df1[pos] = df1[start]
    df2[pos] = df2[end] - endtrim
    return pandas.concat([df1, df2]).sort_values([start, end])

Ignore deprecation warnings from mainly seaborn:

In [28]:
# silence deprecation warnings (lots from seaborn)
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=numpy.VisibleDeprecationWarning)
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

Import variables global to the entire analysis:

In [29]:
import analysis_globals as ag

Local code in the scripts dir on the cluster:

In [30]:
scripts_dir = ag.root_dir / 'scripts'
if str(scripts_dir) not in sys.path:
    sys.path.append(str(scripts_dir))

import simons_meta_data
import hg19_chrom_sizes

## Convenience functions

In [31]:
def silent_nanmean(x):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        return numpy.nanmean(x)
    
def ident_scalar(s):
    x = s.unique()
    assert(len(x)) == 1
    return x[0]

def flatten_column_index(df):
    df.columns = ['_'.join(col).strip() for col in df.columns.values]


## Load meta data

In [32]:
# easy loading of meta data in a consistent manner across code
meta_data_dir = Path(os.environ['HOME'], 'simons/faststorage/data/metadata')
individuals, populations, regions = simons_meta_data.get_meta_data(meta_data_dir=meta_data_dir)

pop_categories = pandas.read_hdf(str(ag.results_dir / 'population_categories.store'), 'sr')
region_categories = pandas.read_hdf(str(ag.results_dir / 'region_categories.store'), 'sr')
# region_colors = dict(zip(list(region_categories), 
#                          ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', 
#                           '#ff7f00', '#ffff33', '#a65628']))

region_colors = dict(zip(list(region_categories), 
                         ['#e41a1c', '#377eb8',  '#984ea3', '#4daf4a',
                          '#ff7f00', '#ffff33', '#a65628']))

chromosome_lengths = dict((k.replace('chr', ''), v) for k, v in hg19_chrom_sizes.hg19_chrom_sizes.items())

# -------------------------------------------------

# -------------------------------------------------

# -------------------------------------------------

# Problem: How can a window be swept if mean_dist is NaN, AND  WHAT IS WITH THE NAN IN ADMIX PROP?

In [25]:
merged_data.loc[(merged_data.swept_af == True) & merged_data.mean_dist_af.isnull()].loc[:, 
                    ['indiv_1', 'start', 'swept', 'swept_af',
                     'mean_dist', 'mean_dist_af', 'admix_prop']]


,indiv_1,start,swept,swept_af,mean_dist,mean_dist_af,admix_prop


In [26]:
merged_data.loc[(merged_data.swept == True) & merged_data.mean_dist.isnull()].loc[:, 
                    ['indiv_1', 'start', 'swept', 'swept_af', 
                     'mean_dist', 'mean_dist_af', 'admix_prop']]


,indiv_1,start,swept,swept_af,mean_dist,mean_dist_af,admix_prop


In [27]:
import sys
sys.path.append('../scripts')

import os, sys
import numpy
import pandas
from pandas import DataFrame, Series
import argparse
from multiprocessing import Pool, cpu_count

import gc

import simons_meta_data
import hg19_chrom_sizes

import networkx as nx
import itertools

In [29]:
#S_Brahmin-1	76900000

df = pandas.read_hdf(root_dir / 'steps/male_dist_admix_masked_stores/male_dist_data_chrX_100kb_twice.hdf')
all_male_dist_twice = df.loc[(df.start >= 76500000) & (df.start <= 77500000)]
del df
gc.collect()

871

In [30]:
from importlib import reload
import clique_calling

In [31]:
reload(clique_calling)
call_rolling_windows = clique_calling.call_rolling_windows
call_swept = clique_calling.call_swept
_window_stats = clique_calling._window_stats

# clique_calling.call_rolling_windows(all_male_dist_twice.loc[(df.start >= 76500000) & (df.start <= 77500000)], 5e-5, 25)


In [32]:
nr_wins = 5
offset = 100000
offsets = [x * offset for x in range(nr_wins)]
window_size = len(offsets) * offset

#all_male_dist_twice = pandas.read_hdf(args.dist_file_name)

# print('all_male_dist_twice', "Ust_Ishim" in all_male_dist_twice.indiv_1.unique())

# remove unused columns
to_keep = ['indiv_1', 'indiv_2', 'start', 'end', 'dist', 'dist_af', 
           'region_1', 'region_2', 'pop_1', 'region_label_1', 'region_id_1', 'uncalled']
to_drop = [x for x in all_male_dist_twice.columns if x not in to_keep]
all_male_dist_twice.drop(to_drop, axis=1, inplace=True)
gc.collect()

nr_indiv = all_male_dist_twice.indiv_1.unique().size

MIN_SWEEP_CLADE_SIZE = round(nr_indiv * 25 / 100)

In [33]:
# partial version of window_stats
import functools
window_stats = functools.partial(_window_stats, pwdist_cutoff=5e-5, min_sweep_clade_size=MIN_SWEEP_CLADE_SIZE)

# merge window sweep info with distance data
if 'dist_af' in all_male_dist_twice.columns:
    # this is not a simulation
    gr_cols = ['indiv_1', 'start', 'end', 'pop_1', 'region_label_1', 'region_id_1', 'region_1']
else:
    gr_cols = ['indiv_1', 'start', 'end']
stats_data = (all_male_dist_twice
        .groupby(gr_cols)
        .apply(window_stats)
        .reset_index(level=gr_cols)
        )

# print('stats_data', "Ust_Ishim" in stats_data.indiv_1.unique())

In [34]:
stats_data.loc[stats_data.indiv_1 == 'S_Brahmin-1']

,indiv_1,start,end,pop_1,region_label_1,region_id_1,region_1,mean_dist,mean_dist_af,mean_dist_to_africans,mean_dist_to_africans_af,prop_indivs_missing,win_swept,win_swept_af
0,S_Brahmin-1,76500000,76600000,Brahmin,SouthAsia,2,SouthAsia,0.001157,0.001160,0.000999,0.001038,0.0,False,False
0,S_Brahmin-1,76600000,76700000,Brahmin,SouthAsia,2,SouthAsia,NaN,NaN,NaN,NaN,1.0,False,False
0,S_Brahmin-1,76700000,76800000,Brahmin,SouthAsia,2,SouthAsia,0.001066,0.000066,0.001050,0.000484,0.0,False,True
0,S_Brahmin-1,76800000,76900000,Brahmin,SouthAsia,2,SouthAsia,0.001023,NaN,0.001010,NaN,0.0,False,False
0,S_Brahmin-1,76900000,77000000,Brahmin,SouthAsia,2,SouthAsia,0.000924,NaN,0.000897,NaN,0.0,False,False
0,S_Brahmin-1,77000000,77100000,Brahmin,SouthAsia,2,SouthAsia,0.000875,NaN,0.000875,NaN,0.0,False,False
0,S_Brahmin-1,77100000,77200000,Brahmin,SouthAsia,2,SouthAsia,0.000354,NaN,0.000389,NaN,0.0,False,False
0,S_Brahmin-1,77200000,77300000,Brahmin,SouthAsia,2,SouthAsia,0.000527,NaN,0.000535,NaN,0.0,False,False
0,S_Brahmin-1,77300000,77400000,Brahmin,SouthAsia,2,SouthAsia,0.000546,NaN,0.000541,NaN,0.0,False,False
0,S_Brahmin-1,77400000,77500000,Brahmin,SouthAsia,2,SouthAsia,0.000936,NaN,0.000937,NaN,0.0,False,False


In [35]:
def call_rolling_windows(df, pwdist_cutoff, min_sweep_clade_size):
    """
    Input data frame holds windows for all individuals in a 500kb window.
    """

#     """
#     dist values may be nan if there are too many uncalled bases, but may also
#     be nan if there is no unmasked bases left after admixture masking. We only
#     want to call the 500kb (and the constituent 100kb windows) as swept if all
#     five 100kb windows are not uncalled. But we want to allow nans if these are
#     induced by admixture masking.
#     """
    
#     def mean_dist(df, dist_col):
#         if (df.uncalled > analysis_globals.max_uncalled_bases).any():
#             # there are uncalled windows in this 500kb so we report 
#             # it as nan to indicdate it cannot be called as swept
#             return numpy.nan
#         else:
#             # there are no uncalled windows so we ingnore any windows 
#             # with nan dist that are induced by missing data after
#             # admixture filtering
#             if df[dist_col].isnull().sum() > 2:
#                 # but return nan if there are more than TWO such windows
#                 return numpy.nan
#             return df[dist_col].mean()


    # build data frame to return
    cols = ['indiv_1', 'start', 'end', 'off']
    result_df = pandas.DataFrame(list(df.groupby(cols).groups.keys()), columns=cols)

    ## NEW: #############################

#     # Get mean dist between pairs in the 500kb window:
#     win_dist = df.groupby(['indiv_1', 'indiv_2']).apply(mean_dist, 'dist')

    ###############################

    # Get mean dist between pairs in the 500kb window (make mean nan if any 100kb is nan):
    win_dist = df.groupby(['indiv_1', 'indiv_2']).dist.agg(numpy.mean)
    
    ###############################

    # Build graph
    graph = nx.Graph()
    for tup in win_dist[win_dist <= pwdist_cutoff].reset_index().itertuples():
        graph.add_edge(tup.indiv_1, tup.indiv_2)
        
    cliques = sorted(nx.algorithms.clique.find_cliques(graph), key=len)

    # find swept
    result_df['called'] = False
    result_df['mean_clade_dist'] = numpy.nan
    result_df['clade_size'] = numpy.nan
    
    for clique in cliques:

        if len(clique) >= min_sweep_clade_size:
            
            # get indiv pairs. (in just one orientation). that is fine because df has both orientations

            called = result_df.indiv_1.isin(clique)
            #result_df['called'] = called | result_df.called # do not make it False if it is already True   
            
            result_df.loc[called, 'called'] = True
            
            # clade_size is the size of the clique
            result_df.loc[called, 'clade_size'] = len(clique) 
            
            # mean_clade_dist is the mean dist of the clique
            indiv_pairs = list(itertools.combinations(sorted(clique), 2))
            result_df.loc[called, 'mean_clade_dist'] = win_dist.loc[indiv_pairs].mean()                


    if 'dist_af' in df.columns:

        #### NEW: #########################

#         # Get mean dist between pairs in the 500kb window (make mean nan if any 100kb is nan):
#         win_dist = df.groupby(['indiv_1', 'indiv_2']).apply(mean_dist, 'dist_af')

        #############################
        
        # Get mean dist between pairs in the 500kb window (make mean nan if any 100kb is nan):
        win_dist = df.groupby(['indiv_1', 'indiv_2']).dist_af.agg(numpy.mean)

        #############################

        # Build graph
        graph = nx.Graph()
        for tup in win_dist[win_dist <= pwdist_cutoff].reset_index().itertuples():
            graph.add_edge(tup.indiv_1, tup.indiv_2)

        cliques = sorted(nx.algorithms.clique.find_cliques(graph), key=len)

        # find swept
        result_df['called_af'] = False
        result_df['mean_clade_dist_af'] = numpy.nan
        result_df['clade_size_af'] = numpy.nan

        for clique in cliques:

            if len(clique) >= min_sweep_clade_size:
                # get indiv pairs. (in just one orientation). that is fine because df has both orientations

                called = result_df.indiv_1.isin(clique)

                result_df.loc[called, 'called_af'] = True

                # clade_size is the size of the clique
                result_df.loc[called, 'clade_size_af'] = len(clique) 

                # mean_clade_dist is the mean dist of the clique
                indiv_pairs = list(itertools.combinations(sorted(clique), 2))
                result_df.loc[called, 'mean_clade_dist_af'] = win_dist.loc[indiv_pairs].mean()

    return result_df


lst = list()
# loop over five offsets of 500kb windows
for off in offsets:
    
    if not off == 300000:
        continue
    
    print(off)
    groups = (all_male_dist_twice
                .assign(off=off, # keep offset
                        roll_win = lambda df: (off + df.start) // window_size) # label for rolling 500kb window
                .groupby(['roll_win', 'off'], as_index=False)

              ### ADDED THIS TO ENSURE NO CALLS ARE MADE ON WINDOWS SMALLER THAN 500KB:
                .filter(lambda df: df.start.unique().size == nr_wins)             
                .groupby(['roll_win', 'off'], as_index=False)

                )
    # with Pool(nr_cpu) as p:
    #     df = pandas.concat(p.map(call_rolling_windows, [group for name, group in groups]))
    ##### added pwdist arg to function call

    lst.append(groups.apply(call_rolling_windows, 5e-5, MIN_SWEEP_CLADE_SIZE))

300000


In [36]:
groups.groups.keys()

dict_keys([(154, 300000)])

In [37]:
df = groups.get_group((154, 300000))

In [38]:
cols = ['indiv_1', 'start', 'end', 'off']
result_df = pandas.DataFrame(list(df.groupby(cols).groups.keys()), columns=cols)

In [39]:
df.loc[df.indiv_1 == 'S_Brahmin-1', ['indiv_1', 'start', 'dist_af']].drop_duplicates()

,indiv_1,start,dist_af
8869007,S_Brahmin-1,76700000,0.000000
8869076,S_Brahmin-1,76700000,NaN
8869080,S_Brahmin-1,76700000,0.010638
8869168,S_Brahmin-1,76800000,NaN
8869329,S_Brahmin-1,76900000,NaN
8869490,S_Brahmin-1,77000000,NaN
8869651,S_Brahmin-1,77100000,NaN


In [40]:
numpy.mean(df.dist_af.values), df.dist_af.isnull().sum(), df.dist_af.mean(), df.dist_af.mean(skipna=False)

(nan, 1294, 0.00015988816, nan)

In [43]:
%load_ext Cython

In [46]:
%%cython
# Get mean dist between pairs in the 500kb window (make mean nan if any 100kb is nan):
#win_dist = df.groupby(['indiv_1', 'indiv_2']).dist_af.agg(numpy.mean)
#
import numpy
cimport numpy

def foo(x):
#    return x.mean(skipna=False)
    return numpy.mean(x.values)

In [48]:
%%timeit
# win_dist = df.groupby(['indiv_1', 'indiv_2']).dist_af.agg(foo)
win_dist = df.groupby(['indiv_1', 'indiv_2']).dist_af.agg(lambda sr: numpy.mean(sr.values))

#win_dist.loc['S_Brahmin-1']

959 ms ± 8.73 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [73]:
#############################

# Build graph
graph = nx.Graph()
for tup in win_dist[win_dist <= 5e-5].reset_index().itertuples():
    graph.add_edge(tup.indiv_1, tup.indiv_2)

cliques = sorted(nx.algorithms.clique.find_cliques(graph), key=len)

# find swept
result_df['called_af'] = False
result_df['mean_clade_dist_af'] = numpy.nan
result_df['clade_size_af'] = numpy.nan

for clique in cliques:

    if len(clique) >= MIN_SWEEP_CLADE_SIZE:
        # get indiv pairs. (in just one orientation). that is fine because df has both orientations
        print('S_Brahmin-1' in clique, clique)
        called = result_df.indiv_1.isin(clique)

        result_df.loc[called, 'called_af'] = True

        # clade_size is the size of the clique
        result_df.loc[called, 'clade_size_af'] = len(clique) 

        # mean_clade_dist is the mean dist of the clique
        indiv_pairs = list(itertools.combinations(sorted(clique), 2))
        result_df.loc[called, 'mean_clade_dist_af'] = win_dist.loc[indiv_pairs].mean()
result_df

True ['S_Brahmin-1', 'S_Bulgarian-2', 'S_Brahui-1', 'S_Punjabi-1', 'S_Even-2', 'S_Oroqen-1', 'B_Sardinian-3', 'S_Polish-1', 'S_Kusunda-2', 'S_Hawaiian-1', 'S_Balochi-2', 'S_Papuan-11', 'S_Estonian-1', 'S_Tajik-1', 'S_Yemenite_Jew-2', 'S_Altaian-1', 'S_Dai-2', 'S_Han-2', 'S_Ami-2', 'S_Yadava-1', 'S_Eskimo_Chaplin-1', 'S_Papuan-10', 'S_Madiga-1', 'S_Karitiana-1', 'S_Orcadian-1', 'S_Lahu-2', 'S_Mala-3', 'S_Yakut-2', 'S_Maori-1', 'S_Tajik-2', 'S_Ami-1', 'S_Pima-1', 'S_Khonda_Dora-1', 'S_Spanish-1', 'S_Iranian-2', 'S_North_Ossetian-2', 'S_Samaritan-1', 'S_Sindhi-1', 'S_Tujia-1', 'S_Saami-2']
True ['S_Brahmin-1', 'S_Bulgarian-2', 'S_Brahui-1', 'S_Punjabi-1', 'S_Even-2', 'S_Oroqen-1', 'B_Sardinian-3', 'S_Polish-1', 'S_Kusunda-2', 'S_Hawaiian-1', 'S_Balochi-2', 'S_Papuan-11', 'S_Estonian-1', 'S_Tajik-1', 'S_Yemenite_Jew-2', 'S_Altaian-1', 'S_Dai-2', 'S_Han-2', 'S_Ami-2', 'S_Yadava-1', 'S_Eskimo_Chaplin-1', 'S_Papuan-10', 'S_Madiga-1', 'S_Karitiana-1', 'S_Orcadian-1', 'S_Lahu-2', 'S_Mala-3', 'S

,indiv_1,start,end,off,called_af,mean_clade_dist_af,clade_size_af
0,B_Australian-4,76700000,76800000,300000,True,0.000031,62.0
1,B_Australian-4,76800000,76900000,300000,True,0.000031,62.0
2,B_Australian-4,76900000,77000000,300000,True,0.000031,62.0
3,B_Australian-4,77000000,77100000,300000,True,0.000031,62.0
4,B_Australian-4,77100000,77200000,300000,True,0.000031,62.0
5,B_Crete-2,76700000,76800000,300000,False,NaN,NaN
6,B_Crete-2,76800000,76900000,300000,False,NaN,NaN
7,B_Crete-2,76900000,77000000,300000,False,NaN,NaN
8,B_Crete-2,77000000,77100000,300000,False,NaN,NaN
9,B_Crete-2,77100000,77200000,300000,False,NaN,NaN


In [68]:
#df.loc[(df.indiv_1 == 'S_Brahmin-1') & (df.indiv_2 == 'S_She-2')]
#df.loc[df.indiv_1 == 'S_Brahmin-1']
df.loc[(df.indiv_1 == 'S_Brahmin-1') & ~df.dist_af.isnull(), ['indiv_1', 'start']]#.sort_values(by='start').drop_duplicates()

,indiv_1,start
8869007,S_Brahmin-1,76700000
8869008,S_Brahmin-1,76700000
8869009,S_Brahmin-1,76700000
8869010,S_Brahmin-1,76700000
8869011,S_Brahmin-1,76700000
8869012,S_Brahmin-1,76700000
8869013,S_Brahmin-1,76700000
8869014,S_Brahmin-1,76700000
8869015,S_Brahmin-1,76700000
8869016,S_Brahmin-1,76700000


In [58]:
test = call_rolling_windows(df,  5e-5, MIN_SWEEP_CLADE_SIZE)
test.loc[test.indiv_1 == 'S_Brahmin-1']
# del all_male_dist_twice
# gc.collect()

,indiv_1,start,end,off,called,mean_clade_dist,clade_size,called_af,mean_clade_dist_af,clade_size_af
175,S_Brahmin-1,76700000,76800000,300000,False,NaN,NaN,True,0.000031,62.0
176,S_Brahmin-1,76800000,76900000,300000,False,NaN,NaN,True,0.000031,62.0
177,S_Brahmin-1,76900000,77000000,300000,False,NaN,NaN,True,0.000031,62.0
178,S_Brahmin-1,77000000,77100000,300000,False,NaN,NaN,True,0.000031,62.0
179,S_Brahmin-1,77100000,77200000,300000,False,NaN,NaN,True,0.000031,62.0


In [44]:
df = pandas.concat(lst)
df.loc[df.indiv_1 == 'S_Brahmin-1']

indiv_1     start       end     off  called  mean_clade_dist  \
0 175  S_Brahmin-1  76500000  76600000       0   False              NaN   
  176  S_Brahmin-1  76600000  76700000       0   False              NaN   
  177  S_Brahmin-1  76700000  76800000       0   False              NaN   
  178  S_Brahmin-1  76800000  76900000       0   False              NaN   
  179  S_Brahmin-1  76900000  77000000       0   False              NaN   
1 175  S_Brahmin-1  77000000  77100000       0   False              NaN   
  176  S_Brahmin-1  77100000  77200000       0   False              NaN   
  177  S_Brahmin-1  77200000  77300000       0   False              NaN   
  178  S_Brahmin-1  77300000  77400000       0   False              NaN   
  179  S_Brahmin-1  77400000  77500000       0   False              NaN   
0 175  S_Brahmin-1  76900000  77000000  100000   False              NaN   
  176  S_Brahmin-1  77000000  77100000  100000   False              NaN   
  177  S_Brahmin-1  77100000  77200000  100000   False              NaN   
  178  S_Brahmin-1  77200000  77300000  100000   False              NaN   
  179  S_Brahmin-1  77300000  77400000  100000   False              NaN   
  175  S_Brahmin-1  76800000  76900000  200000   False              NaN   
  176  S_Brahmin-1  76900000  77000000  200000   False              NaN   
  177  S_Brahmin-1  77000000  77100000  200000   False              NaN   
  178  S_Brahmin-1  77100000  77200000  200000   False              NaN   
  179  S_Brahmin-1  77200000  77300000  200000   False              NaN   
  175  S_Brahmin-1  76700000  76800000  300000   False              NaN   
  176  S_Brahmin-1  76800000  76900000  300000   False              NaN   
  177  S_Brahmin-1  76900000  77000000  300000   False              NaN   
  178  S_Brahmin-1  77000000  77100000  300000   False              NaN   
  179  S_Brahmin-1  77100000  77200000  300000   False              NaN   
  175  S_Brahmin-1  76600000  76700000  400000   False              NaN   
  176  S_Brahmin-1  76700000  76800000  400000   False              NaN   
  177  S_Brahmin-1  76800000  76900000  400000   False              NaN   
  178  S_Brahmin-1  76900000  77000000  400000   False              NaN   
  179  S_Brahmin-1  77000000  77100000  400000   False              NaN   
1 175  S_Brahmin-1  77100000  77200000  400000   False              NaN   
  176  S_Brahmin-1  77200000  77300000  400000   False              NaN   
  177  S_Brahmin-1  77300000  77400000  400000   False              NaN   
  178  S_Brahmin-1  77400000  77500000  400000   False              NaN   
  179  S_Brahmin-1  77500000  77600000  400000   False              NaN   

       clade_size  called_af  mean_clade_dist_af  clade_size_af  
0 175         NaN      False                 NaN            NaN  
  176         NaN      False                 NaN            NaN  
  177         NaN      False                 NaN            NaN  
  178         NaN      False                 NaN            NaN  
  179         NaN      False                 NaN            NaN  
1 175         NaN      False                 NaN            NaN  
  176         NaN      False                 NaN            NaN  
  177         NaN      False                 NaN            NaN  
  178         NaN      False                 NaN            NaN  
  179         NaN      False                 NaN            NaN  
0 175         NaN      False                 NaN            NaN  
  176         NaN      False                 NaN            NaN  
  177         NaN      False                 NaN            NaN  
  178         NaN      False                 NaN            NaN  
  179         NaN      False                 NaN            NaN  
  175         NaN      False                 NaN            NaN  
  176         NaN      False                 NaN            NaN  
  177         NaN      False                 NaN            NaN  
  178         NaN      False                 NaN        

In [41]:
# concatenate data frames for each offset and call windows as swept
sweep_calls = (pandas.concat(lst)
                .groupby(['indiv_1', 'start', 'end'])
                .apply(call_swept)
                .reset_index(level=['indiv_1', 'start', 'end'])
                )

# del lst[:]
# gc.collect()

In [42]:
sweep_calls.loc[sweep_calls.indiv_1 == 'S_Brahmin-1']

,indiv_1,start,end,called,called_af,clade_mean_dist,clade_mean_dist_af,clade_size,clade_size_af
0,S_Brahmin-1,76500000,76600000,False,False,NaN,NaN,NaN,NaN
0,S_Brahmin-1,76600000,76700000,False,True,NaN,0.000030,NaN,59.0
0,S_Brahmin-1,76700000,76800000,False,True,NaN,0.000031,NaN,62.0
0,S_Brahmin-1,76800000,76900000,False,True,NaN,0.000031,NaN,62.0
0,S_Brahmin-1,76900000,77000000,False,True,NaN,0.000031,NaN,62.0
0,S_Brahmin-1,77000000,77100000,False,True,NaN,0.000031,NaN,62.0
0,S_Brahmin-1,77100000,77200000,False,True,NaN,0.000031,NaN,62.0
0,S_Brahmin-1,77200000,77300000,False,False,NaN,NaN,NaN,NaN
0,S_Brahmin-1,77300000,77400000,False,False,NaN,NaN,NaN,NaN
0,S_Brahmin-1,77400000,77500000,False,False,NaN,NaN,NaN,NaN


In [ ]:
# print('sweep_calls', "Ust_Ishim" in sweep_calls.indiv_1.unique())

sweep_data = stats_data.merge(sweep_calls, on=['indiv_1', 'start', 'end'])

# del stats_data
# del sweep_calls
# gc.collect()

# print('sweep_data', "Ust_Ishim" in sweep_data.indiv_1.unique())

# get run length of swept windows and call windows as part of sweeps (ECH haplotypes)
def run_id(sr):
    return (sr != sr.shift()).cumsum()

sweep_data['run_id'] = (sweep_data
                        .groupby('indiv_1')['called']
                        .apply(run_id)
                       )
sweep_data['run_length'] = (sweep_data
                            .groupby(['indiv_1', 'run_id'])['run_id']
                            .transform(numpy.size)
                           )
sweep_data['swept'] = numpy.bitwise_and(sweep_data['called'], 
                                        sweep_data['run_length'] >= analysis_globals.min_run_length)

if 'called_af' in sweep_data.columns:
    # this is not a simulation
    sweep_data['run_id_af'] = (sweep_data
                            .groupby('indiv_1')['called_af']
                            .apply(run_id)
                        )
    sweep_data['run_length_af'] = (sweep_data
                                .groupby(['indiv_1', 'run_id_af'])['run_id_af']
                                .transform(numpy.size)
                            )
    sweep_data['swept_af'] = numpy.bitwise_and(sweep_data['called_af'], 
                                            sweep_data['run_length_af'] >= analysis_globals.min_run_length)

gc.collect()

# print('sweep_data', "Ust_Ishim" in sweep_data.indiv_1.unique())

# write to hdf output file
sweep_data.to_hdf(args.sweep_data_file_name, 'df', mode='w', format='table')

(286902, 19)

------

# WHAT IS WITH THE NAN IN ADMIX PROP FOR B_Ju_hoan_North-4 above?

Assuming the latter are all swept, they make up three percent of all the swept windows:

In [32]:
len(new_uncallable_when_admix_filtered) / ((merged_data.swept_af == True).sum() + len(new_uncallable_when_admix_filtered))

0.03064938381967946

and since they are all archiac, that would add three percent points to the admixture proportion in swept windows. That is ofcause completely unrealistic, but shows that this way of doing admixture filtering does not help me show "admixture displacement" is not just an arteract of not being able to call sweeps that overlap admixture.

Ideas:

- I could look at only windows with short admix segments (so dist does not become nan). I would compare match and match_af to see how much of a window that is masked and then only include windows where only a little is masked...
- I could also modify the calling to allow one of the five windows to be nan

The latter is probably the most robust idea. It would not affect the analysisi without admixture filtering. It would require that I somehow record if a dist is nan because uncalled is too high or becuase there is not data left after admxture filtering

In [33]:
new_uncallable_when_admix_filtered.admix_prop.unique(), new_uncallable_when_admix_filtered.admix_prop.isnull().sum()

(array([ 1., nan]), 384)

# -------------------------------------------------

# -------------------------------------------------

# -------------------------------------------------